<a href="https://colab.research.google.com/github/ajdhiman/Deep-Learning/blob/main/DL_Classification_Model_(Cat_%26_Dog).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request as request
from zipfile import ZipFile
from tqdm import tqdm
import os
import tensorflow  as tf

Downloading Data

In [2]:
remoteSourcePath = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"
localZipFilePath = "data.zip"
localPath = "Data"
localPathForClasses= "Data/PetImages"

In [3]:
filename, headers = request.urlretrieve(url=remoteSourcePath, filename=localZipFilePath)

In [4]:
def get_updated_list_of_files(list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

def preProcess(zf: ZipFile, f: str, working_dir: str):

    target_filepath = os.path.join(working_dir, f)
    if not os.path.exists(target_filepath):
        zf.extract(f, working_dir)

    if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

In [5]:
with ZipFile(file=localZipFilePath, mode="r") as zf:
  list_of_files = zf.namelist()
  updated_list_of_files = get_updated_list_of_files(list_of_files)

  print(len(updated_list_of_files))
  for f in tqdm(updated_list_of_files):
      preProcess(zf, f, localPath)


25000


100%|██████████| 25000/25000 [00:10<00:00, 2362.15it/s]


Getting VGG16 Model

In [6]:
model = tf.keras.applications.vgg16.VGG16(
            input_shape=[224, 224, 3],
            weights="imagenet",
            include_top=False
        )

model.save("VGG16.h5")

58889256/58889256 [==============================] - 0s 0us/step


In [7]:
def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

In [8]:
fullModel = prepare_full_model(model, 2, True, None, 0.01)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
def train_valid_generator():

        dataGenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=[224, 224],
            batch_size=16,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **dataGenerator_kwargs
        )

        valid_generator = valid_datagenerator.flow_from_directory(
            directory=localPathForClasses,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


        train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=40,
            horizontal_flip=True,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            **dataGenerator_kwargs
        )

        train_generator = train_datagenerator.flow_from_directory(
            directory=localPathForClasses,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        return train_generator,valid_generator

train_generator,valid_generator = train_valid_generator()

Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [10]:
train_generator,valid_generator = train_valid_generator()

Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [11]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size

fullModel.fit(
    train_generator,
    epochs=1,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    validation_data=valid_generator
)


  27/1250 [..............................] - ETA: 5:02 - loss: 13.0155 - accuracy: 0.5579

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1250/1250 [==============================] - 350s 271ms/step - loss: 6.2497 - accuracy: 0.6865 - val_loss: 1.0625 - val_accuracy: 0.8998


In [12]:
fullModel.save("DLClassification.h5")

In [13]:
def _valid_generator():

  datagenerator_kwargs = dict(
    rescale = 1./255,
    validation_split=0.30
  )

  dataflow_kwargs = dict(
    target_size=[224,224],
    batch_size=16,
    interpolation="bilinear"
  )

  valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagenerator_kwargs
  )

  valid_generator = valid_datagenerator.flow_from_directory(
    directory=localPathForClasses,
    subset="validation",
    shuffle=True,
    **dataflow_kwargs
  )
  return  valid_generator


In [14]:
valid_generator= _valid_generator()

Found 7498 images belonging to 2 classes.


In [15]:
fullModel.evaluate(valid_generator)

469/469 [==============================] - 38s 82ms/step - loss: 1.0337 - accuracy: 0.8992


[1.0336908102035522, 0.8991731405258179]